In [1]:
# for article content scripting with BeautifulSoup
import requests
from bs4 import BeautifulSoup
import re

# for rendering html template with Jinja
from jinja2 import Environment, FileSystemLoader
import os

# for building and sending emails with MailChimp
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import json

# for viewing the email draft in webpage
import webbrowser
from datetime import date

import config as cf
import importlib
cf = importlib.reload(cf)

# Define the Article Object with BeautifulSoup

In [49]:
class Article():
    
    def __init__(self, url, title=None, img_url=None, preview_text=None, tag=None, label=None, class_name='article', id_name='main', video_page_url=None):
        
        self._url = url.replace('/gb/', '/b5/')
        self._title = title
        self._img_url = img_url
        self._preview_text = preview_text
        self._tag = tag
        self._label = label
        self._video_page_url = video_page_url.replace('/gb/', '/b5/')
        
        page = requests.get(self._url)
        self._soup = BeautifulSoup(page.text, 'html.parser')
        
        self._article = self._soup.find(class_=class_name)
        
        if not self._article:
            self._article = self._soup.find(id=id_name)
            
        if not self._article:
            self._article = self._soup.find('article')
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        
    @property
    def title(self, class_name='title'):
        
        if not self._title:
            article_title = self._article.find(class_=class_name)
            self._title = article_title.text
            
        return self._title
    
    @title.setter
    def title(self, value):
        self._title = value
    
    @property
    def img_url(self, class_name='featured_image'):
        
        if not self._img_url:
            try:
                self._img_url = self._article.find(class_=class_name).find('a').get('href')
            except:
                try:
                    self._img_url = self._article.find(class_=class_name).find('figure').find('img').get('src')
                except:
                    try:
                        self._img_url = self._article.find(class_='wp-post-image').get('src')
                    except:
                        page = requests.get(self._video_page_url)
                        video_page_soup = BeautifulSoup(page.text, 'html.parser')
                        self._img_url = video_page_soup.select_one(f"a[href='{self._url}']").find('img').get('src')
        
        return self._img_url
    
    @img_url.setter
    def img_url(self, value):
        self._img_url = value
    
    @property
    def preview_text(self, id_name='artbody'):
        
        paragraph_index = 0
        
        while not self._preview_text:
            article_first_paragraph = self._article.find(id=id_name).find_all('p')[paragraph_index].text
            article_preview_text = re.sub(r'\【[^)]*\】', '', article_first_paragraph)
            article_preview_text = re.sub(r'\n', '', article_preview_text) 
            article_preview_text = re.sub(r'\（英文大纪元[^)]*\编译）', '', article_preview_text)
            article_preview_text = re.sub(r'\（英文大紀元[^)]*\編譯）', '', article_preview_text)
            article_preview_text = re.sub(r'\（英國[^)]*\報導）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            article_preview_text = re.sub(r'\(大紀元[^)]*\報導\)', '', article_preview_text)
            article_preview_text = re.sub(r'\（大纪元[^)]*\报导）', '', article_preview_text)
            self._preview_text = article_preview_text.strip()
            paragraph_index += 1
        
        return self._preview_text
    
    @preview_text.setter
    def preview_text(self, value):
        self._preview_text = value
        
    @property
    def tag(self, id_name='breadcrumb'):
        
        if not self._tag:
            breadcrumb = self._soup.find(id=id_name)
            raw_tag = breadcrumb.findAll('a')[-1].text
            raw_tag = re.sub(r'\([^)]*\)', '', raw_tag)
            self._tag = re.sub(r'\（[^)]*\）', '', raw_tag)
        
        return self._tag
    
    @tag.setter
    def tag(self, value):
        self._tag = value
        
    @property
    def label(self):
        return self._label
    
    @label.setter
    def label(self, value):
        self._label = value

In [50]:
class Content(Article):
    
    def __init__(self, article_url_list, video_url_list, video_page_url=None):
        
        # parse data
        self.article_list = [Article(article_url, video_page_url=video_page_url) for article_url in article_url_list]
        
        self.video_list = [Article(video_url, video_page_url=video_page_url) for video_url in video_url_list]
        self.video_pair_list = [[value, self.video_list[counter+1]] for counter, value in enumerate(self.video_list) if counter%2 == 0]      

In [51]:
class Jinja_Template(Content):
    
    def __init__(self, 
                 template_folder_name = 'Jinja Templates', 
                 template_file_name = 'newsletter_as_jinja_template_base.html'):

        file_loader = FileSystemLoader('../'+template_folder_name+'/')
        self.env = Environment(loader=file_loader, extensions=['jinja2.ext.do'])

        # render the base template
        self.template = self.env.get_template(template_file_name)
        
    def render(self, source_contect, 
               folder_name='Exported HTML', 
               rendered_newsletter_filename = 'rendered_newsletter.html'):
        
        # description: subject_line and preview_text
        self.subject_line = source_contect.article_list[0].title
        article_title_list = [article.title for article in source_contect.article_list]
        self.preview_text = ' · '.join(article_title_list[1:])
        
        self.rendered_content = self.template.render(env=self.env, 
                                                   header=source_contect.article_list[0], 
                                                   article_list=source_contect.article_list, 
                                                   video_pair_list=source_contect.video_pair_list)
        
        self.folder_path = '../' + folder_name + '/'
        
        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)
        
        self.newsletter_path = self.folder_path + rendered_newsletter_filename
        
        with open(self.newsletter_path, "w") as fh:
            fh.write(self.rendered_content)
            
        print('newsletter rendered at: '+ self.newsletter_path)
        
    def preview(self):

        webbrowser.open('file://' + os.path.realpath(self.newsletter_path))

In [52]:
class MailChimp_Campaign(Jinja_Template):

    def __init__(self, newsletter, mailchimp_config,
                 # sender information
                 from_name='Anonymous Sender',
                 reply_to='no_reply@email.com'):

        self.subject_line = newsletter.subject_line
        self.preview_text = newsletter.preview_text
        self.rendered_content = newsletter.rendered_content
        self.mailchimp_config = mailchimp_config

        # sender information
        self.from_name = from_name
        self.reply_to = reply_to
        
        # campaign name
        today = date.today().strftime("%d %B %Y")
        self.campaign_title = 'Campaign_Created_through_API' + today

    def connect_to_MailChimp(self):

        self.client = MailchimpMarketing.Client()
        self.client.set_config({
            "api_key": self.mailchimp_config["API_KEY"],
            "server": self.mailchimp_config["API_KEY"].split('-')[-1]
        })

        response_ping = self.client.ping.get()

        if response_ping['health_status'] == "Everything's Chimpy!":
            print("MailChimp account connected!")

    def update_MailChimp_template(self):

        response = self.client.templates.update_template(
            self.mailchimp_config['template_id'], {"name": "Template_Created_through_API", "html": self.rendered_content})
        print("Template updated at:", response['date_edited'])

    def specify_recipients(self):

        if self.mailchimp_config['segment_id']:
            self.recipients = {'segment_opts':
                               {'saved_segment_id':
                                   self.mailchimp_config['segment_id']},
                               'list_id': self.mailchimp_config['list_id']}
        else:
            self.recipients = {'list_id': self.mailchimp_config['list_id']}

        print('recipients specified.')

    def create(self):

        self.connect_to_MailChimp()
        self.update_MailChimp_template()
        self.specify_recipients()

        self.created_campaign = self.client.campaigns.create({"type": "regular",
                                                              'recipients': self.recipients,
                                                              "settings":
                                                              {"title": self.campaign_title,
                                                               "subject_line": self.subject_line,
                                                               "preview_text": self.preview_text,
                                                               "from_name": self.from_name,
                                                               "reply_to": self.reply_to,
                                                               "template_id": self.mailchimp_config['template_id'],
                                                               "auto_footer": False}})
        print('Campaign created successfully!')

    def send(self):

        campaign_id = self.created_campaign['id']
        response_of_campaigns_send = self.client.campaigns.send(campaign_id)

        return response_of_campaigns_send

# Processing Part

In [53]:
article_url_list = ['https://www.epochtimes.com/b5/21/5/22/n12967665.htm',
                    'https://www.epochtimes.com/b5/21/5/20/n12963653.htm',
                    'https://www.epochtimes.com/b5/21/5/20/n12963673.htm',
                    'https://www.epochtimes.com/b5/21/5/20/n12963647.htm']

video_url_list = ['https://www.epochtimes.com/b5/21/5/19/n12961559.htm',
                  'https://www.epochtimes.com/b5/21/5/19/n12961300.htm',
                  'https://www.epochtimes.com/b5/21/5/20/n12964086.htm',
                  'https://www.ntdtv.com/b5/2021/05/23/a103126080.html']

video_page_url = 'https://www.epochtimes.com/gb/ncid1247351.htm'

In [54]:
source_content = Content(article_url_list, video_url_list, video_page_url=video_page_url)

In [55]:
for article in source_content.article_list:
    print(article.title)
    print(article.tag)
    print(article.label)
    print(article.preview_text)
    print()

組圖：英國首相約翰遜訪問伊麗莎白女王號
國際要聞
None
週五（5月21日），英國首相鮑里斯．約翰遜（Boris Johnson）至樸次茅斯（Portsmouth）訪問伊麗莎白女王號（HMS Queen Elizabeth）航空母艦，週六（22日）伊麗莎白女王號將進行遠洋部署，預計前往約40個國家，並與多國軍隊進行聯合演習。

強硬派當選DUP領袖 北愛局勢更難料
政治新聞
None
近日，強硬派普茨（Edwin Poots）當選北愛最大親英國黨派民主統一黨（DUP）領袖。由於他在脫歐等問題上的立場，北愛的局勢可能變得更加難以預料。

英國高端百貨Selfridges將主辦婚禮
英倫生活
None
近日，英國高端百貨商店塞爾弗里奇（Selfridges）獲得執照，可以在倫敦牛津街的旗艦店為顧客舉行婚禮。

英國多所大學網課持續至秋季
教育新聞
None
近日，英國多所大學表示，他們會繼續為學生提供網絡課程，這種情況會一直持續到今年9月開始的新學年的第一個學期。許多學生非常氣憤，要求學校退還學費。



In [110]:
# source_content.article_list[0].title = '疫情导致去年英国经济缩水创300年记录'

In [9]:
for article in source_content.video_list:
    try:
        print(article.title)
        print(article.tag)
        print(article.preview_text)
        print(article.img_url)
        print()
    except:
        pass

【有冇搞錯】杭州失蹤的金錢豹哪去了？
時政評論
《有冇搞錯》。5月20日。
https://i.epochtimes.com/assets/uploads/2021/05/id12963238-68c14447541d1ed1b03a771b00ded154-320x200.jpg

【探索時分】太平洋定海神針 美救台的第七艦隊
軍事天地
1950年6月28日台灣基隆港外，一艘軍艦緩緩駛來，你仔細一看，艦首兩門兩聯裝的127毫米火炮，舷號DD-745，這不是美國驅逐艦「布拉什號」（USS Brush）嗎？美國驅逐艦為什麼要到台灣來？
https://i.epochtimes.com/assets/uploads/2021/05/id12961305-ea6c2c3d9bf92975378a456fcdbdfeff-320x200.jpg

【未解之謎】水往高處流 探祕「神祕點」
神秘地帶
大家好，我是扶搖。
https://i.epochtimes.com/assets/uploads/2021/05/id12964160-1200x800-2-320x200.jpg

【慶豐茶館】騙子扎堆兒了，茶館要被公私合營了？
時事小品


In [11]:
source_content.video_list[3].img_url = 'https://i.ntdtv.com/assets/uploads/2021/05/maxresdefault-56-360x202.jpg'

In [12]:
for video in source_content.video_list:
    print(video.img_url)

https://i.epochtimes.com/assets/uploads/2021/05/id12963238-68c14447541d1ed1b03a771b00ded154-320x200.jpg
https://i.epochtimes.com/assets/uploads/2021/05/id12961305-ea6c2c3d9bf92975378a456fcdbdfeff-320x200.jpg
https://i.epochtimes.com/assets/uploads/2021/05/id12964160-1200x800-2-320x200.jpg
https://i.ntdtv.com/assets/uploads/2021/05/maxresdefault-56-360x202.jpg


In [13]:
newsletter = Jinja_Template()

In [14]:
newsletter.render(source_content)

newsletter rendered at: ../Exported HTML/rendered_newsletter.html


In [15]:
newsletter.preview()

# Connect to MailChimp

## Test Campaign

In [16]:
mailchimp_config_test = {
    "API_KEY": cf.mailchimp_config_test.API_KEY,
    "template_id": cf.mailchimp_config_test.template_id,
    "segment_id": cf.mailchimp_config_test.segment_id, # recipients
    "list_id": cf.mailchimp_config_test.list_id # members list
}

### Create MailChimp campaign

In [17]:
test_campaign = MailChimp_Campaign(newsletter, mailchimp_config_test, 
                                   from_name=cf.mailchimp_compaign.from_name, 
                                   reply_to=cf.mailchimp_compaign.reply_to)

In [18]:
test_campaign.create()

MailChimp account connected!
Template updated at: 2021-05-23T17:46:09+00:00
recipients specified.
Campaign created successfully!


In [19]:
test_campaign.send()

<Response [204]>

## Real Campaign

In [20]:
mailchimp_config_real = {
    "API_KEY": cf.mailchimp_config_real.API_KEY,
    "template_id": cf.mailchimp_config_real.template_id,
    "segment_id": cf.mailchimp_config_real.segment_id, # recipients
    "list_id": cf.mailchimp_config_real.list_id # members list
}

In [21]:
real_campaign = MailChimp_Campaign(newsletter, mailchimp_config_real, 
                                   from_name=cf.mailchimp_compaign.from_name, 
                                   reply_to=cf.mailchimp_compaign.reply_to)

In [22]:
real_campaign.create()

MailChimp account connected!
Template updated at: 2021-05-23T17:46:24+00:00
recipients specified.
Campaign created successfully!


In [23]:
real_campaign.send()

<Response [204]>

## Drafts

In [200]:
# define article tag
tag_list = ['英国新闻', '英国新闻', '英国新闻', '英国新闻']

for index in range(len(tag_list)):
    article_list[index].tag = tag_list[index]
    
# article_list[0].img_url = 'https://i.epochtimes.com/assets/uploads/2020/12/2012112056442124.jpg'